In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pkg_resources

from debvader.training.train import train_deblender

2024-11-05 03:27:08.606860: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-05 03:27:08.630501: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-05 03:27:09.122390: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Download and format data

Download dataset that will be used for training. It has been generated using the code in https://github.com/BastienArcelin/dc2_img_generation and the stamps are generated using the XXX function see notebook XXX. The size of the stamp is then fixed to 59x59 pixels.

In [2]:
data_folder_path = pkg_resources.resource_filename('debvader', "data/")
image_path = os.path.join(data_folder_path + '/dc2_imgs/imgs_dc2.npy')
images = np.load(image_path, mmap_mode = 'c')

In [3]:
images.shape

(10, 59, 59, 6)

Separate training data and labels to feed the VAE and deblender. You should be careful that the number of filter in the data for training correspond to the number of bands considered for the network (default is six).

In [4]:
training_data_vae = np.array((images[:5], images[:5]))
validation_data_vae = np.array((images[5:], images[5:]))

training_data_deblender = np.array((images[:5], images[:5]))
validation_data_deblender = np.array((images[5:], images[5:]))

### Train the VAE and deblender with architecture from the debvader paper (https://arxiv.org/abs/2005.12039)

In [5]:
hist_vae, hist_deblender, net = train_deblender("lsst",
                                                      from_survey = "dc2", 
                                                      epochs = 100, 
                                                      training_data_vae = training_data_vae, 
                                                      validation_data_vae = validation_data_vae, 
                                                      training_data_deblender = training_data_deblender, 
                                                      validation_data_deblender = validation_data_deblender)

2024-11-05 03:27:10.141123: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1094 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6
2024-11-05 03:27:10.142343: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 16708 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:08:00.0, compute capability: 8.6


in cropping
VAE model
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 59, 59, 6)]       0         
                                                                 
 model (Functional)          (None, 560)               3741224   
                                                                 
 multivariate_normal_tri_l   ((None, 32),              0         
 (MultivariateNormalTriL)     (None, 32))                        
                                                                 
 model_1 (Functional)        (None, 59, 59, 6)         4577228   
                                                                 
Total params: 8318452 (31.73 MB)
Trainable params: 8318440 (31.73 MB)
Non-trainable params: 12 (48.00 Byte)
_________________________________________________________________
/home/parlange/debvader/src/debvader/data/weights/dc2

Start 

2024-11-05 03:27:12.918315: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
2024-11-05 03:27:13.256323: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 - 4s - loss: 2197132.5000 - mse: 0.0797 - kl_metric: 0.2512 - val_loss: 1194038.7500 - val_mse: 0.0997 - val_kl_metric: 0.2393 - 4s/epoch - 4s/step
Epoch 2/100
1/1 - 0s - loss: 1162577.0000 - mse: 0.0917 - kl_metric: 0.2415 - val_loss: 413581.8750 - val_mse: 0.1054 - val_kl_metric: 0.2310 - 58ms/epoch - 58ms/step
Epoch 3/100
1/1 - 0s - loss: 802102.5000 - mse: 0.1051 - kl_metric: 0.2875 - val_loss: 43965.2461 - val_mse: 0.1105 - val_kl_metric: 0.2602 - 81ms/epoch - 81ms/step
Epoch 4/100
1/1 - 0s - loss: 21846.5625 - mse: 0.1268 - kl_metric: 0.2917 - val_loss: 17877.9199 - val_mse: 0.1292 - val_kl_metric: 0.2683 - 29ms/epoch - 29ms/step
Epoch 5/100
1/1 - 0s - loss: 6792.0928 - mse: 0.1269 - kl_metric: 0.3384 - val_loss: 11135.1973 - val_mse: 0.1181 - val_kl_metric: 0.2364 - 30ms/epoch - 30ms/step
Epoch 6/100
1/1 - 0s - loss: 3873.1946 - mse: 0.1341 - kl_metric: 0.3573 - val_loss: 7804.7095 - val_mse: 0.1597 - val_kl_metric: 0.2864 - 52ms/epoch - 52ms/step
Epoch 7/100
1/1 - 0s - loss

### Dataset with a different number of filters

The number of available filters is different for each survey. For example, five filters are available for DES data. To change that you should specify it in the train_deblender function, as well as if the channels appear last or first in the data array.

In [6]:
training_data_vae.shape

(2, 5, 59, 59, 6)

Here the channels appear last in the data array. So the channel_first option in the train_deblender function is set as False (default setting).

In [7]:
hist_vae, hist_deblender, net = train_deblender("des",
                                                      from_survey = "dc2", 
                                                      epochs = 100, 
                                                      training_data_vae = training_data_vae, 
                                                      validation_data_vae = validation_data_vae, 
                                                      training_data_deblender = training_data_deblender, 
                                                      validation_data_deblender = validation_data_deblender,
                                                      nb_of_bands = 6,
                                                      channel_last = True)

in cropping
VAE model
Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 59, 59, 6)]       0         
                                                                 
 model_4 (Functional)        (None, 560)               3741224   
                                                                 
 multivariate_normal_tri_l_  ((None, 32),              0         
 1 (MultivariateNormalTriL)   (None, 32))                        
                                                                 
 model_5 (Functional)        (None, 59, 59, 6)         4577228   
                                                                 
Total params: 8318452 (31.73 MB)
Trainable params: 8318440 (31.73 MB)
Non-trainable params: 12 (48.00 Byte)
_________________________________________________________________
/home/parlange/debvader/src/debvader/data/weights/dc2

Start 

We need to change the training data format. For example just train on the five fist channels.

In [10]:
training_data_vae_deslike = np.array((images[:5,:,:,:5], images[:5,:,:,:5]))
validation_data_vae_deslike = np.array((images[5:,:,:,:5], images[5:,:,:,:5]))

training_data_deblender_deslike = np.array((images[:5,:,:,:5], images[:5,:,:,:5]))
validation_data_deblender_deslike = np.array((images[5:,:,:,:5], images[5:,:,:,:5]))

Now try the training:

Warning: we cannot use the weights from the network trained on DC2 as the data has six channels in that case. We need to set the from_survey option to None.

In [11]:
hist_vae, hist_deblender, net = train_deblender("des",
                                                      from_survey = None, 
                                                      epochs = 100, 
                                                      training_data_vae = training_data_vae_deslike, 
                                                      validation_data_vae = validation_data_vae_deslike, 
                                                      training_data_deblender = training_data_deblender_deslike, 
                                                      validation_data_deblender = validation_data_deblender_deslike,
                                                      nb_of_bands = 5,
                                                      channel_last = True)

in cropping
VAE model
Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 59, 59, 5)]       0         
                                                                 
 model_12 (Functional)       (None, 560)               3740932   
                                                                 
 multivariate_normal_tri_l_  ((None, 32),              0         
 3 (MultivariateNormalTriL)   (None, 32))                        
                                                                 
 model_13 (Functional)       (None, 59, 59, 5)         4576650   
                                                                 
Total params: 8317582 (31.73 MB)
Trainable params: 8317572 (31.73 MB)
Non-trainable params: 10 (40.00 Byte)
_________________________________________________________________

Start the training
Epoch 1/100
1/1 - 2s - loss: 2135851.250